In [2]:
from recipe_scrapers import scrape_me

scraper = scrape_me("https://www.marmiton.org/recettes/recette_muffins-tres-simples_166385.aspx")
scraper.title()
scraper.instructions()
scraper.to_json()
# for a complete list of methods:
# help(scraper)

{'author': 'Anonyme',
 'canonical_url': 'https://www.marmiton.org/recettes/recette_muffins-tres-simples_166385.aspx',
 'category': 'muffin',
 'cook_time': 15,
 'cuisine': 'Dessert',
 'description': 'farine, oeuf, sucre, lait, beurre, levure chimique, sucre vanillé, sel, cannelle, extrait de vanille, eau',
 'host': 'marmiton.org',
 'image': 'https://assets.afcdn.com/recipe/20221020/136482_w1024h576c1cx3370cy2350cxt0cyt0cxb6720cyb4480.jpg',
 'ingredient_groups': [{'ingredients': ['280 g de farine',
    '2 oeufs',
    '100 g de sucre',
    '150 ml de lait',
    "80 g de beurre (ou 8 cuillères à soupe d'huile)",
    '1 sachets de levure chimique',
    '1 sachets de sucre vanillé',
    '1 pincées de sel',
    'cannelle',
    'extrait de vanille',
    'eau de vie, etc..)'],
   'purpose': None}],
 'ingredients': ['280 g de farine',
  '2 oeufs',
  '100 g de sucre',
  '150 ml de lait',
  "80 g de beurre (ou 8 cuillères à soupe d'huile)",
  '1 sachets de levure chimique',
  '1 sachets de sucre v

In [3]:
from recipe_scrapers import scrape_me

# 1. On récupère toutes les données de la page
scraper = scrape_me("https://www.marmiton.org/recettes/recette_muffins-tres-simples_166385.aspx")

# 2. On sélectionne uniquement ce qui nous intéresse
# On les stocke dans un dictionnaire pour que ce soit propre
ma_recette_filtree = {
    "titre": scraper.title(),
    "ingredients": scraper.ingredients(),
    "instructions": scraper.instructions()
}

print(ma_recette_filtree)

{'titre': 'Muffins très simples', 'ingredients': ['280 g de farine', '2 oeufs', '100 g de sucre', '150 ml de lait', "80 g de beurre (ou 8 cuillères à soupe d'huile)", '1 sachets de levure chimique', '1 sachets de sucre vanillé', '1 pincées de sel', 'cannelle', 'extrait de vanille', 'eau de vie, etc..)'], 'instructions': "Dans un premier saladier : mélanger la farine, le sucre, le sel, le sucre vanillé et la levure.\nDans un deuxième saladier : mélanger le lait, l'huile(ou beurre fondu) et les oeufs.\nVerser le contenu du second saladier dans le premier et remuer jusqu'à obtenir une pâte plutôt lisse. Laissez tout de même quelques petits grumeaux.\nMettre au four 15 min à 180°C (thermostat 6)"}


In [4]:
import requests
from bs4 import BeautifulSoup
from recipe_scrapers import scrape_me
import time

def obtenir_recettes_muffins(nb_pages=1):
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=muffin"
    liens_recettes = []
    base_de_donnees = []

    # --- ÉTAPE A : Récupérer les liens ---
    print("Recherche des liens de muffins...")
    for page in range(1, nb_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Sur Marmiton, les liens de recettes sont souvent dans des balises <a> avec une classe spécifique
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and "/recettes/recette_" in href:
                full_url = "https://www.marmiton.org" + href if href.startswith('/') else href
                if full_url not in liens_recettes:
                    liens_recettes.append(full_url)

    # --- ÉTAPE B : Scraper chaque lien ---
    print(f"{len(liens_recettes)} recettes trouvées. Début de l'extraction...")
    for url in liens_recettes:
        try:
            scraper = scrape_me(url)
            
            # On crée ton dictionnaire personnalisé
            donnees = {
                'author': scraper.author(),
                'canonical_url': url,
                'category': 'muffin', # On le force car on sait ce qu'on cherche
                'cook_time': scraper.total_time(), # Temps total en minutes
                'ingredients': scraper.ingredients(),
                'instructions': scraper.instructions()
            }
            base_de_donnees.append(donnees)
            print(f"Récupéré : {donnees['titre']}")
            
            # Petite pause pour ne pas être bloqué par le site (important !)
            time.sleep(1) 
            
        except Exception as e:
            print(f"Erreur sur {url}: {e}")

    return base_de_donnees

# Lancer la création sur la première page de résultats
ma_bdd = obtenir_recettes_muffins(nb_pages=1)



Recherche des liens de muffins...
21 recettes trouvées. Début de l'extraction...
Erreur sur https://www.marmiton.org/recettes/recette_muffins-de-sandrine_57367.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-a-la-banane_14745.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-poires-pepites-de-chocolat-de-nadine_24727.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-a-la-framboise-de-jeanne_37909.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-nutella-et-pepites-de-chocolat_57089.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-coeur-de-nutella_57023.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-aux-pommes_11582.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-aux-pepites-de-chocolat_20636.aspx: 'titre'
Erreur sur https://www.marmiton.org/recettes/recette_muffins-aux-framboises_39470.aspx: 'titre'
Erreur

In [5]:
import requests
from bs4 import BeautifulSoup
from recipe_scrapers import scrape_me
import time

def obtenir_vrais_muffins(nb_pages=1):
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=muffin"
    base_de_donnees = []
    headers = {'User-Agent': 'Mozilla/5.0'}

    for page in range(1, nb_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # On récupère tous les liens de recettes sur la page
        liens = [a.get('href') for a in soup.select('a[href*="/recettes/recette_"]')]
        
        for href in set(liens): # 'set' permet d'éviter les doublons de liens
            full_url = "https://www.marmiton.org" + href if href.startswith('/') else href
            
            try:
                scraper = scrape_me(full_url)
                titre = scraper.title()
                
                # --- LA CONDITION DE FILTRE ---
                # .lower() transforme tout en minuscules pour ne pas rater "Muffin" avec une majuscule
                if "muffin" in titre.lower():
                    recette = {
                        'title': titre,
                        'author': scraper.author(),
                        'canonical_url': full_url,
                        'category': scraper.category(), # On reprend la vraie catégorie du site
                        'ingredients': scraper.ingredients(),
                        'instructions': scraper.instructions(),
                    }
                    base_de_donnees.append(recette)
                    print(f"✅ Ajouté : {titre}")
                else:
                    print(f"❌ Ignoré (pas un muffin) : {titre}")
                
                time.sleep(0.5)

            except Exception as e:
                print(f"⚠️ Erreur sur {full_url} : {e}")

    return base_de_donnees

# Lancement du script
ma_liste_muffins = obtenir_vrais_muffins(1)
print(f"\nTerminé ! Tu as récupéré {len(ma_liste_muffins)} vraies recettes de muffins.")

✅ Ajouté : Muffins à la pâte à tartiner et aux pépites de chocolat
✅ Ajouté : Muffins très simples
❌ Ignoré (pas un muffin) : Salade de pâtes au thon
✅ Ajouté : Muffins aux pommes
✅ Ajouté : Muffins coeur de Nutella
✅ Ajouté : Muffins aux framboises
✅ Ajouté : Muffins aux pépites de chocolat
❌ Ignoré (pas un muffin) : Ratatouille
✅ Ajouté : Muffins au jambon et olives vertes de Nadine
❌ Ignoré (pas un muffin) : Caviar d'aubergines
❌ Ignoré (pas un muffin) : Quiche aux poireaux et lardons
❌ Ignoré (pas un muffin) : Salade de riz d'été facile
✅ Ajouté : Muffins à la banane
✅ Ajouté : Muffins surprises au chocolat
✅ Ajouté : Muffins pomme cannelle gingembre
✅ Ajouté : Muffins à la framboise de Jeanne
❌ Ignoré (pas un muffin) : Houmous (purée de pois chiches)
✅ Ajouté : Muffins de Sandrine
✅ Ajouté : Muffins allégés aux pépites de chocolat
✅ Ajouté : Muffins poires pépites de chocolat de Nadine
✅ Ajouté : Muffins framboises et chocolat blanc

Terminé ! Tu as récupéré 15 vraies recettes de 

In [7]:
import requests
from bs4 import BeautifulSoup
from recipe_scrapers import scrape_me
import time

def scraper_tout_marmiton(recherche="muffin"):
    base_url = f"https://www.marmiton.org/recettes/recherche.aspx?aqt={recherche}"
    base_de_donnees = []
    page = 1
    continuer = True
    headers = {'User-Agent': 'Mozilla/5.0'}

    while continuer:
        print(f"--- Analyse de la page {page} ---")
        url_page = f"{base_url}&page={page}"
        response = requests.get(url_page, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # On cherche les liens de recettes
        liens = [a.get('href') for a in soup.select('a[href*="/recettes/recette_"]')]
        
        # SI AUCUN LIEN TROUVÉ : On a atteint la fin des résultats
        if not liens:
            print("Fin des résultats atteinte.")
            continuer = False
            break

        # On nettoie les doublons de la page avec set()
        for href in set(liens):
            full_url = "https://www.marmiton.org" + href if href.startswith('/') else href
            
            try:
                scraper = scrape_me(full_url)
                titre = scraper.title()
                
                # On garde ton filtre "muffin" dans le titre
                if "muffin" in titre.lower():
                    recette = {
                        'title': titre,
                        'url': full_url,
                        'ingredients': scraper.ingredients(),
                        'instructions': scraper.instructions()
                    }
                    base_de_donnees.append(recette)
                    print(f"✅ {titre}")
                
                time.sleep(0.4) # Un peu plus rapide mais toujours prudent

            except Exception as e:
                # Si une recette plante, on passe à la suivante sans arrêter tout le script
                continue

        # Passage à la page suivante
        page += 1
        
        # SÉCURITÉ : On s'arrête à 20 pages max pour tes tests (tu peux augmenter)
        if page > 20: 
            continuer = False

    return base_de_donnees

# Lancement
ma_super_base = scraper_tout_marmiton("muffin")

--- Analyse de la page 1 ---
✅ Muffins à la pâte à tartiner et aux pépites de chocolat
✅ Muffins très simples
✅ Muffins aux pommes
✅ Muffins coeur de Nutella
✅ Muffins aux framboises
✅ Muffins aux pépites de chocolat
✅ Muffins au jambon et olives vertes de Nadine
✅ Muffins à la banane
✅ Muffins surprises au chocolat
✅ Muffins pomme cannelle gingembre
✅ Muffins à la framboise de Jeanne
✅ Muffins de Sandrine
✅ Muffins allégés aux pépites de chocolat
✅ Muffins poires pépites de chocolat de Nadine
✅ Muffins framboises et chocolat blanc
--- Analyse de la page 2 ---
✅ muffins au thon et aux olives
✅ muffins choco-banane
✅ Muffins aux myrtilles
✅ Muffins à la banane faciles
✅ Muffins aux fraises et à la menthe
✅ Muffins salés poulet-curry
✅ Mini muffins choco-noisettes
✅ Muffins pommes canelle
✅ Muffins aux pépites de chocolat, noix et cannelle
✅ Muffins tout chocolat
✅ Muffins au miel
✅ Muffin bananes et pépites de chocolat
✅ Mini-muffins fondants à l'orange
✅ Mes muffins banane chocolat bla

In [8]:
import json

# Supposons que ta liste s'appelle 'ma_liste_muffins'
with open('ma_super_base.json', 'w', encoding='utf-8') as f:
    json.dump(ma_super_base, f, indent=4, ensure_ascii=False)

print("Fichier base_muffins.json créé !")

Fichier base_muffins.json créé !


In [ ]:
import json
with open('muffins_db.json', 'w', encoding='utf-8') as f:
    json.dump(ma_bdd, f, indent=4, ensure_ascii=False)

In [11]:
import requests
from bs4 import BeautifulSoup
from recipe_scrapers import scrape_me
import time

def scraper_atelier_chefs_fixed(recherche="muffin"):
    # L'URL de recherche correcte
    base_url = f"https://www.atelierdeschefs.fr/fr/recherche/recettes/{recherche}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    base_de_donnees = []

    try:
        response = requests.get(base_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Le sélecteur spécifique pour l'Atelier des Chefs :
        # On cherche les balises <a> qui contiennent "/recette/" dans leur lien
        # et qui sont à l'intérieur des cartes de résultats
        liens = []
        for a in soup.select('a[href*="/fr/recette/"]'):
            href = a['href']
            # On s'assure d'avoir l'URL complète
            full_url = "https://www.atelierdeschefs.fr" + href if href.startswith('/') else href
            
            # On évite les doublons et les liens inutiles (ex: liens de réseaux sociaux)
            if full_url not in liens and "/fr/recette/" in full_url:
                liens.append(full_url)

        if not liens:
            print("Toujours 0 liens. Il est possible que le site bloque les requêtes simples.")
            return []

        print(f"Trouvé {len(liens)} recettes potentielles. Analyse en cours...")

        for url in liens:
            try:
                scraper = scrape_me(url)
                titre = scraper.title()
                
                if recherche.lower() in titre.lower():
                    base_de_donnees.append({
                        'title': titre,
                        'url': url,
                        'ingredients': scraper.ingredients(),
                        'instructions': scraper.instructions()
                    })
                    print(f"✅ {titre}")
                time.sleep(0.5)
            except:
                continue

    except Exception as e:
        print(f"Erreur lors de la requête : {e}")

    return base_de_donnees

# Test
resultats = scraper_atelier_chefs_fixed("muffin")

Toujours 0 liens. Il est possible que le site bloque les requêtes simples.
